In [4]:

from algorithm.order_plan.multi_machines.object.static_parameter import StaticParameters
from algorithm.order_plan.multi_machines.data_process import DataProcess

data_process = DataProcess()

# 合同数据
orders = data_process.get_all_orders_data_frame()

orders_obj_list = data_process.get_all_orders() # 获取订单对象列表

orders_obj_list = [
    order for order in orders_obj_list
    if order.plan_start_date.year == 2024 and order.plan_start_date.month == 11
]

orders_obj_list = orders_obj_list[:10]

# 静态参数
static_params = StaticParameters()



2025-11-09 19:13:35,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-09 19:13:35,833 INFO sqlalchemy.engine.Engine SELECT "TOCAUSP"."NO" AS "TOCAUSP_NO", "TOCAUSP"."ORDER_NO" AS "TOCAUSP_ORDER_NO", "TOCAUSP"."PROD_CODE_TYPE" AS "TOCAUSP_PROD_CODE_TYPE", "TOCAUSP"."PLAN_COMMENT" AS "TOCAUSP_PLAN_COMMENT", "TOCAUSP"."PROD_CPIN" AS "TOCAUSP_PROD_CPIN", "TOCAUSP"."DELIVY_DATE" AS "TOCAUSP_DELIVY_DATE", "TOCAUSP"."ORDER_MONTH_PRO" AS "TOCAUSP_ORDER_MONTH_PRO", "TOCAUSP"."ORDER_MONTH" AS "TOCAUSP_ORDER_MONTH", "TOCAUSP"."SG_SIGN" AS "TOCAUSP_SG_SIGN", "TOCAUSP"."ST_NO" AS "TOCAUSP_ST_NO", "TOCAUSP"."SURF_QUALITY_GRADE" AS "TOCAUSP_SURF_QUALITY_GRADE", "TOCAUSP"."ORDER_THICK" AS "TOCAUSP_ORDER_THICK", "TOCAUSP"."ORDER_WIDTH" AS "TOCAUSP_ORDER_WIDTH", "TOCAUSP"."LACK_WT" AS "TOCAUSP_LACK_WT", "TOCAUSP"."ORDER_WT" AS "TOCAUSP_ORDER_WT", "TOCAUSP"."SURF_STRUC_DESC" AS "TOCAUSP_SURF_STRUC_DESC", "TOCAUSP"."SURF_STRUC_CODE" AS "TOCAUSP_SURF_STRUC_CODE", "TOCAUSP"."ORDER_TYPE_CODE" AS "TO

In [5]:
import pandas as pd
import copy
import math
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
from algorithm.order_plan.multi_machines.object.order_multi_feature import OrderMultiFeature
from datetime import datetime, timedelta
import random
from deap import base, creator, tools, algorithms
import itertools

# ------------------------------------------------------------------------------------------------------
# 成本计算函数
# ------------------------------------------------------------------------------------------------------

def calculate_adjacency_cost(order1: OrderMultiFeature, order2: OrderMultiFeature) -> float:
    """
    计算两个相邻生产的订单之间的成本。
    主要考虑宽度递减、厚度集中原则，以及宽度反跳的惩罚。
    成本值越大表示越不优，越小表示越优。
    """
    cost = 0.0

    # 1. 宽度递减原则
    width_diff = order1.order_width - order2.order_width
    if width_diff < 0:
        # 宽度增加 (反跳)
        if abs(width_diff) < 50: # 小反跳，不建议，但不是重开批次
            cost += 50.0
        else: # 大反跳，相当于重开一个批次，成本更高
            cost += 200.0
    else:
        # 宽度递减或保持，鼓励
        cost -= 10.0 # 略微奖励

    # 2. 厚度集中原则
    thick_diff = abs(order1.order_thick - order2.order_thick)
    if thick_diff > 10.0: # 厚度差异大，惩罚
        cost += thick_diff * 5.0
    else: # 厚度差异小，鼓励
        cost -= 5.0 # 略微奖励

    # 3. 如果是相同订单号，则成本为0 (尽管在实际生产中可能不会相邻)
    if order1.order_no == order2.order_no:
        return 0.0

    return max(0.0, cost) # 确保成本非负

# 预计算所有订单对之间的成本，以便快速查询
order_adjacency_costs = {}
for i, order1 in enumerate(orders_obj_list):
    for j, order2 in enumerate(orders_obj_list):
        if i != j:
            order_adjacency_costs[(order1.order_no, order2.order_no)] = calculate_adjacency_cost(order1, order2)
        else:
            order_adjacency_costs[(order1.order_no, order2.order_no)] = 0.0 # 同一个订单相邻生产，成本为0

# ------------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------------

from datetime import datetime, timedelta
import random
from deap import base, creator, tools

# ----------------------------- 辅助函数 --------------------------------


def hours_to_timedelta(hours: float) -> timedelta:
    return timedelta(seconds=round(hours * 3600))



def processing_time_hours(order: OrderMultiFeature, op: str, machine_idx: int, static: StaticParameters) -> float:
    speed = static.speed[op][machine_idx]
    return order.order_wt / speed



# 计算一次完整的调度结果
@dataclass
class Task:
    order_no: str
    op: str
    machine_idx: int
    start: datetime
    end: datetime

@dataclass
class ScheduleResult:
    tasks: List[Task]
    # 方便查询：schedule[(order_no, op)] = Task
    index: Dict[Tuple[str, str], Task]


# 评估函数：返回目标值（越小越好）和违约/库存信息
@dataclass
class EvalResult:
    objective: float
    tardiness_total_hours: float
    adjacency_cost: float
    inventory_penalty: float
    stock_violations: Dict[str, float] # op -> 超出量（吨）正为超上限，负为低于下限


# ----------------------------- 排程生成 --------------------------------

def build_greedy_schedule(orders: List[OrderMultiFeature], static: StaticParameters, order_sequence: Optional[List[str]] = None) -> ScheduleResult:
    """
    构建初始贪心调度：按order_sequence（order_no列表）顺序逐单调度其所有工序。
    每道工序选择能尽早完成（finish time 最早）的机组。
    """
    if order_sequence is None:
        # 按交付日期（EDD）排序
        orders_sorted = sorted(orders, key=lambda o: o.delivery_date)
    else:
        order_map = {o.order_no: o for o in orders}
        orders_sorted = [order_map[ono] for ono in order_sequence]

    # 记录每台机组的可用时间（op, machine_idx) -> datetime
    machine_available: Dict[Tuple[str, int], datetime] = {}
    # 初始可用时间为每个订单最早的 plan_start_date 的最小值（或更合理为最早plan_start_date）
    global_earliest = min(o.plan_start_date for o in orders)
    for op in static.operation_sequence:
        for mi in range(len(static.speed[op])):
            machine_available[(op, mi)] = global_earliest

    tasks: List[Task] = []
    index: Dict[Tuple[str, str], Task] = {}

    for order in orders_sorted:
        # arrival time to first op is >= order.plan_start_date
        arrival = order.plan_start_date
        for op_idx, op in enumerate(static.operation_sequence):
            best_machine = None
            best_start = None
            best_end = None
            for mi in range(len(static.speed[op])):
                ready = max(machine_available[(op, mi)], arrival)
                pt_hours = processing_time_hours(order, op, mi, static)
                finish = ready + hours_to_timedelta(pt_hours)
                if best_end is None or finish < best_end:
                    best_end = finish
                    best_start = ready
                    best_machine = mi
            # 记录任务
            t = Task(order_no=order.order_no, op=op, machine_idx=best_machine, start=best_start, end=best_end)
            tasks.append(t)
            index[(order.order_no, op)] = t
            # 更新机组可用时间
            machine_available[(op, best_machine)] = best_end
            # 到达下一工序的时间 = finish + transmission_time[op]
            arrival = best_end + hours_to_timedelta(static.transmission_time[op])

    return ScheduleResult(tasks=tasks, index=index)


# ----------------------------- 评估调度结果 -------------------------------

def evaluate_schedule(schedule: ScheduleResult, orders_map: Dict[str, OrderMultiFeature], static: StaticParameters, stock_penalty_coef: float = 1000.0) -> EvalResult:
    """
    计算目标：拖期小时总和 + 邻接成本 + 库存惩罚
    库存计算：对于每个工序 op，其"后库"序列由在该工序完成并运输到下道工序，但下道工序未开始时的区间累加。
    我们通过收集事件（时间点上库存增减）来精确计算最大库存与最小库存并与 stock_limit 比较。
    """
    # 1) 计算 tardiness（以小时计）
    tardiness_total = 0.0
    # 2) 计算邻接成本：对每台机组上每道工序上同一机组的序列按安排顺序计算邻接成本
    adjacency_cost_total = 0.0

    # 按工序+机组排序任务以计算邻接成本
    tasks_by_op_machine: Dict[Tuple[str, int], List[Task]] = {}
    for t in schedule.tasks:
        tasks_by_op_machine.setdefault((t.op, t.machine_idx), []).append(t)

    for key, tlist in tasks_by_op_machine.items():
        tlist_sorted = sorted(tlist, key=lambda x: x.start)
        for i in range(len(tlist_sorted) - 1):
            o1 = orders_map[tlist_sorted[i].order_no]
            o2 = orders_map[tlist_sorted[i+1].order_no]
            adjacency_cost_total += calculate_adjacency_cost(o1, o2)

    # 计算 tardiness
    # 以最后一道工序的完成时间为交付完成时间
    for order_no, order in orders_map.items():
        last_op = static.operation_sequence[-1]
        t = schedule.index[(order_no, last_op)]
        finish = t.end
        if finish > order.delivery_date:
            tard_hours = (finish - order.delivery_date).total_seconds() / 3600.0
            tardiness_total += tard_hours

    # 计算库存时间线（事件扫描）
    stock_violations: Dict[str, float] = {}
    inventory_penalty = 0.0
    # 对每个工序，构造库存事件：每当某订单在该工序完成并运输到下工序，则在到达时间开始增加；当该订单在下工序开始生产时库存减少
    for op_idx, op in enumerate(static.operation_sequence):
        events: List[Tuple[datetime, float]] = []  # (time, delta_weight)
        for order_no, order in orders_map.items():
            t_finish = schedule.index[(order_no, op)].end
            # 到达时间到下工序后库 = finish + transmission_time
            arrival_to_next = t_finish + hours_to_timedelta(static.transmission_time[op])
            # 下工序开始时间（若存在）
            if op_idx + 1 < len(static.operation_sequence):
                next_op = static.operation_sequence[op_idx + 1]
                t_next_start = schedule.index[(order_no, next_op)].start
            else:
                # 如果没有下工序（最后一站），则以到交期再减少库存
                t_next_start = order.delivery_date
            # 只有当 arrival_to_next < t_next_start 时，该订单会在该后库产生库存
            if arrival_to_next < t_next_start:
                events.append((arrival_to_next, order.order_wt))
                events.append((t_next_start, -order.order_wt))
        # 扫描事件
        events.sort(key=lambda x: (x[0], -x[1]))
        cur = 0.0
        min_stock = float('inf')
        max_stock = float('-inf')
        for time, delta in events:
            cur += delta
            min_stock = min(min_stock, cur)
            max_stock = max(max_stock, cur)
        if min_stock == float('inf'):
            # 没有事件 -> 库存为0
            min_stock = 0.0
            max_stock = 0.0
        lower, upper = static.stock_limit[op]
        # 记录违反量（超上限为正，低于下限为负）——如果都满足则为0
        over = max(0.0, max_stock - upper)
        under = max(0.0, lower - min_stock)
        stock_violations[op] = over - under
        # 库存惩罚按超出体积与违反时间无关的简化惩罚
        inventory_penalty += (over + under) * stock_penalty_coef

    # 目标：我们把 tardiness（小时）和 adjacency_cost 线性组合，再加库存惩罚
    objective = tardiness_total + adjacency_cost_total + inventory_penalty

    return EvalResult(objective=objective, tardiness_total_hours=tardiness_total,
                      adjacency_cost=adjacency_cost_total, inventory_penalty=inventory_penalty,
                      stock_violations=stock_violations)


# ----------------------------- 修复库存违规（启发式） ----------------------

def repair_inventory(schedule: ScheduleResult, orders_map: Dict[str, OrderMultiFeature], static: StaticParameters,
                     max_iter: int = 1000) -> ScheduleResult:
    """
    简单启发式：当库存超上限时，识别造成堆积的订单（按在后库中的重量贡献排序），对这些订单在当前工序的开始时间延后一段时间，直到库存降到上限或达成迭代次数。
    注意：这是启发式操作，可能引入拖期，故后续会由全局搜索进行权衡。
    """
    sched = copy.deepcopy(schedule)
    for it in range(max_iter):
        evalr = evaluate_schedule(sched, orders_map, static)
        # 找到最严重的超上限工序
        worst_op = None
        worst_over = 0.0
        for op, val in evalr.stock_violations.items():
            # stock_violations 为 正值表示超上限（over - under）
            if val > worst_over:
                worst_over = val
                worst_op = op
        if worst_op is None or worst_over <= 1e-6:
            break  # 无明显超出
        # 构建该工序的后库事件细节，找到在该时段内贡献最多的订单
        op_idx = static.operation_sequence.index(worst_op)
        events = []  # (arrival_to_next, t_next_start, order_no, wt)
        for ono, order in orders_map.items():
            t_finish = sched.index[(ono, worst_op)].end
            arrival_to_next = t_finish + hours_to_timedelta(static.transmission_time[worst_op])
            if op_idx + 1 < len(static.operation_sequence):
                next_op = static.operation_sequence[op_idx + 1]
                t_next_start = sched.index[(ono, next_op)].start
            else:
                t_next_start = arrival_to_next
            if arrival_to_next < t_next_start:
                events.append((arrival_to_next, t_next_start, ono, order.order_wt))
        # 按贡献排序（最晚完成的先延后）
        events.sort(key=lambda x: x[0])
        if not events:
            break
        # 选择贡献大且较早到达的订单，延后其在worst_op上的开始
        # 选k个订单尝试延后
        k = max(1, int(len(events) * 0.2))
        candidates = sorted(events, key=lambda x: x[3], reverse=True)[:k]
        changed = False
        for arr, nxtstart, ono, wt in candidates:
            tcur = sched.index[(ono, worst_op)]
            # 延后该任务 start 到 nxtstart - transmission（尽量让其刚好到下道工序开始时）
            target_start = nxtstart - hours_to_timedelta(processing_time_hours(orders_map[ono], worst_op, tcur.machine_idx, static)) - hours_to_timedelta(0.001)
            if target_start > tcur.start:
                # 将任务延后（同时也需要将后续工序相应后移，以保持流动性）
                shift = target_start - tcur.start
                # 对该订单后续工序逐个后移
                opidx = static.operation_sequence.index(worst_op)
                for oi in range(opidx, len(static.operation_sequence)):
                    opi = static.operation_sequence[oi]
                    tt = sched.index[(ono, opi)]
                    tt.start += shift
                    tt.end += shift
                changed = True
                break
        if not changed:
            break
    return sched


# ----------------------------- 模拟退火局部搜索 ----------------------------

def optimize_schedule(orders: List[OrderMultiFeature], static: StaticParameters,
                      time_limit_seconds: int = 30,
                      sa_iters: int = 2000,
                      seed: Optional[int] = None) -> Tuple[ScheduleResult, EvalResult]:
    """
    主入口：贪心生成初始解 -> 修复库存 -> 模拟退火改进（扰动为交换订单序列或改变某工序机组）
    返回最终排程与评估结果
    """
    if seed is not None:
        random.seed(seed)

    orders_map = {o.order_no: o for o in orders}

    # 初始方案
    base_sched = build_greedy_schedule(orders, static)
    base_sched = repair_inventory(base_sched, orders_map, static)
    base_eval = evaluate_schedule(base_sched, orders_map, static)

    best_sched = copy.deepcopy(base_sched)
    best_eval = base_eval

    current_sched = copy.deepcopy(base_sched)
    current_eval = base_eval

    # SA 参数
    T0 = 1.0
    Tmin = 1e-4
    iters = sa_iters

    for it in range(iters):
        T = T0 * ((Tmin / T0) ** (it / max(1, iters - 1)))
        # 产生扰动：随机选择两种扰动-> 1) 交换两个订单在 sequence 中的位置 2) 改变某订单在某道工序的机组
        cand_orders = list(orders_map.keys())
        new_sequence = [t.order_no for t in sorted(current_sched.tasks, key=lambda x: (x.start, x.order_no))]
        # 提供两个扰动策略
        if random.random() < 0.6:
            # 交换两个订单的优先级: 先提取当前序列的唯一订单顺序
            uniq = []
            for ono in new_sequence:
                if ono not in uniq:
                    uniq.append(ono)
            if len(uniq) >= 2:
                i, j = random.sample(range(len(uniq)), 2)
                uniq[i], uniq[j] = uniq[j], uniq[i]
            # 重建排程按新序列
            cand_sched = build_greedy_schedule(orders, static, order_sequence=uniq)
        else:
            # 改变某任务的机组选择，然后从头重建排程（按 EDD 保持顺序）
            # 随机选一个订单和一个工序并随机分配另一个可用机组
            cand_orders = list(orders_map.keys())
            ono = random.choice(cand_orders)
            op = random.choice(static.operation_sequence)
            # 随机将该任务在原始生成时选择的机组替换（但因为我们的 greedy 会重新选择最早完成机组，
            # 这里我们通过暂时打乱 speeds 来影响选择——简单做法：我们再执行 build_greedy 并随后强行修改该任务机器）
            # 为保持逻辑简单，直接对该订单顺序进行小调整：把订单移动到序列头或尾
            uniq = sorted(orders_map.keys(), key=lambda x: random.random())
            if random.random() < 0.5:
                uniq.remove(ono)
                uniq.insert(0, ono)
            else:
                uniq.remove(ono)
                uniq.append(ono)
            cand_sched = build_greedy_schedule(orders, static, order_sequence=uniq)

        # 修复并评估
        cand_sched = repair_inventory(cand_sched, orders_map, static)
        cand_eval = evaluate_schedule(cand_sched, orders_map, static)

        # 接受准则
        delta = cand_eval.objective - current_eval.objective
        if delta < 0 or random.random() < math.exp(-delta / max(1e-9, T)):
            current_sched = cand_sched
            current_eval = cand_eval
            # 更新最优
            if cand_eval.objective < best_eval.objective:
                best_sched = copy.deepcopy(cand_sched)
                best_eval = cand_eval
        # 简单的早停策略
        if it % 200 == 0:
            pass

    return best_sched, best_eval

# -------------------------------------------------------------------------------
# 可视化函数
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from datetime import datetime, timedelta
def plot_gantt_chart(schedule_result: ScheduleResult):
    """
    绘制合同生产甘特图。
    """
    df_gantt = pd.DataFrame([
        dict(
            Task=f"{t.op} - 机组{t.machine_idx+1}",
            Start=t.start,
            Finish=t.end,
            Resource=t.order_no,
            Duration=(t.end - t.start).total_seconds() / 3600
        ) for t in schedule_result.tasks
    ])

    fig = px.timeline(
        df_gantt,
        x_start="Start",
        x_end="Finish",
        y="Task",
        color="Resource",
        title="合同生产甘特图",
        hover_name="Resource",
        hover_data={"Start": "|%Y-%m-%d %H:%M", "Finish": "|%Y-%m-%d %H:%M", "Duration": True, "Task": False}
    )
    fig.update_yaxes(autorange="reversed") # 让最上面的任务显示在图表顶部
    fig.show()

def plot_inventory_change(schedule_result: ScheduleResult, orders_map: Dict[str, OrderMultiFeature], static_params: StaticParameters):
    """
    绘制每个产线的库存变化图。
    """
    fig = make_subplots(rows=len(static_params.operation_sequence), cols=1,
                        shared_xaxes=True,
                        subplot_titles=[f"{op} 后库库存变化" for op in static_params.operation_sequence])

    for i, op in enumerate(static_params.operation_sequence):
        events = []  # (time, delta_weight, order_no, event_type)
        for order_no, order in orders_map.items():
            t_finish = schedule_result.index[(order_no, op)].end
            arrival_to_next = t_finish + hours_to_timedelta(static_params.transmission_time[op])

            op_idx = static_params.operation_sequence.index(op)
            if op_idx + 1 < len(static_params.operation_sequence):
                next_op = static_params.operation_sequence[op_idx + 1]
                t_next_start = schedule_result.index[(order_no, next_op)].start
            else:
                t_next_start = order.delivery_date # 最后一站，视为立即出库

            events.append((arrival_to_next, order.order_wt)) # 只需要时间点和delta
            events.append((t_next_start, -order.order_wt))

        # 扫描事件
        events.sort(key=lambda x: (x[0], -x[1])) # 相同时间点，先处理入库 (+)，后处理出库 (-)

        time_points = []
        stock_levels = []
        current_stock = 0.0

        # 确保包含初始库存
        if events:
            min_event_time = events[0][0]
            time_points.append(min_event_time - timedelta(minutes=1)) # 稍微提前一点
            stock_levels.append(0.0)

        # 使用 itertools.groupby 聚合同一时间点的事件
        for time_key, group in itertools.groupby(events, key=lambda x: x[0]):
            aggregated_delta = sum(item[1] for item in group)

            # 在 time_key 记录事件前的库存
            # 只有当上一个时间点不同时才添加
            if not time_points or time_points[-1] < time_key:
                time_points.append(time_key)
                stock_levels.append(current_stock)

            # 更新库存
            current_stock += aggregated_delta

            # 在 time_key 记录事件后的最终库存
            time_points.append(time_key)
            stock_levels.append(current_stock)


        if not events: # 如果没有库存事件，库存始终为0
            if schedule_result.tasks:
                start_time = min(t.start for t in schedule_result.tasks)
                end_time = max(t.end for t in schedule_result.tasks)
                time_points = [start_time - timedelta(hours=1), end_time + timedelta(hours=1)]
                stock_levels = [0.0, 0.0]
            else:
                time_points = []
                stock_levels = []


        df_stock = pd.DataFrame({"Time": time_points, "Stock": stock_levels})

        if not df_stock.empty:
            fig.add_trace(go.Scatter(x=df_stock["Time"], y=df_stock["Stock"],
                                     mode='lines', name=f'{op} 库存',
                                     hovertemplate='<b>时间:</b> %{x|%Y-%m-%d %H:%M}<br><b>库存:</b> %{y:.2f} 吨<extra></extra>'),
                          row=i+1, col=1)

            lower_limit, upper_limit = static_params.stock_limit[op]
            if not df_stock.empty:
                limit_x = [df_stock["Time"].min(), df_stock["Time"].max()]
            else:
                limit_x = [min(t.start for t in schedule_result.tasks) - timedelta(hours=1),
                           max(t.end for t in schedule_result.tasks) + timedelta(hours=1)] if schedule_result.tasks else [datetime.now(), datetime.now() + timedelta(days=1)]

            fig.add_trace(go.Scatter(x=limit_x, y=[upper_limit]*len(limit_x),
                                     mode='lines', line=dict(dash='dash', color='red'),
                                     name=f'{op} 上限', showlegend=True if i==0 else False,
                                     hovertemplate='<b>库存上限:</b> %{y:.2f} 吨<extra></extra>'),
                          row=i+1, col=1)
            fig.add_trace(go.Scatter(x=limit_x, y=[lower_limit]*len(limit_x),
                                     mode='lines', line=dict(dash='dash', color='green'),
                                     name=f'{op} 下限', showlegend=True if i==0 else False,
                                     hovertemplate='<b>库存下限:</b> %{y:.2f} 吨<extra></extra>'),
                          row=i+1, col=1)
            fig.update_yaxes(title_text="库存量 (吨)", row=i+1, col=1)

    fig.update_layout(title_text="产线库存变化图", height=300 * len(static_params.operation_sequence),
                      hovermode="x unified")
    fig.show()

def plot_tardiness_chart(schedule_result: ScheduleResult, orders_map: Dict[str, OrderMultiFeature], static_params: StaticParameters):
    """
    绘制拖期变化图（有正有负）。
    正值表示拖期，负值表示提前。
    """
    tardiness_data = []
    for order_no, order in orders_map.items():
        last_op = static_params.operation_sequence[-1]
        t = schedule_result.index[(order_no, last_op)]
        finish_time = t.end

        # 计算拖期或提前
        time_diff = (finish_time - order.delivery_date).total_seconds() / 3600.0 # 小时

        tardiness_data.append(dict(
            Order=order_no,
            DeliveryDate=order.delivery_date,
            FinishTime=finish_time,
            TardinessHours=time_diff,
            Status="拖期" if time_diff > 0 else ("提前" if time_diff < 0 else "按时")
        ))

    df_tardiness = pd.DataFrame(tardiness_data)

    fig = px.bar(
        df_tardiness,
        x="Order",
        y="TardinessHours",
        color="Status",
        color_discrete_map={"拖期": "red", "提前": "green", "按时": "blue"},
        title="订单拖期/提前情况",
        labels={"TardinessHours": "拖期/提前小时数", "Order": "订单号"},
        hover_data={"DeliveryDate": "|%Y-%m-%d %H:%M", "FinishTime": "|%Y-%m-%d %H:%M", "TardinessHours": ":.2f"}
    )

    fig.add_hline(y=0, line_dash="dash", line_color="grey") # 添加0刻度线
    fig.update_yaxes(title_text="拖期/提前小时数 (正:拖期, 负:提前)")
    fig.show()


In [6]:
best_sched, best_eval = optimize_schedule(orders_obj_list, static_params, sa_iters=800, seed=42)

print('Objective:', best_eval.objective)
print('Tardiness hours:', best_eval.tardiness_total_hours)
print('Adjacency cost:', best_eval.adjacency_cost)
print('Inventory penalty:', best_eval.inventory_penalty)
print('Stock violations:', best_eval.stock_violations)
print('\nSchedule:')
for t in sorted(best_sched.tasks, key=lambda x: (x.start, x.order_no)):
    print(f"Order {t.order_no} | Op {t.op} | M{t.machine_idx} | {t.start.strftime('%Y-%m-%d %H:%M')} -> {t.end.strftime('%Y-%m-%d %H:%M')}")

# 调用绘图函数
orders_map = {o.order_no: o for o in orders_obj_list}
print("\n绘制甘特图...")
plot_gantt_chart(best_sched)

print("\n绘制库存变化图...")
plot_inventory_change(best_sched, orders_map, static_params)

print("\n绘制拖期变化图...")
plot_tardiness_chart(best_sched, orders_map, static_params)

Objective: 1310720.0
Tardiness hours: 0.0
Adjacency cost: 720.0
Inventory penalty: 1310000.0
Stock violations: {<Operation.HR: 'HR'>: 150.0, <Operation.AZ: 'AZ'>: 150.0, <Operation.LT: 'LT'>: 1010.0}

Schedule:
Order G024012532 | Op Operation.HR | M0 | 2024-11-01 00:00 -> 2024-11-01 02:30
Order G024012526 | Op Operation.HR | M0 | 2024-11-01 02:30 -> 2024-11-01 05:00
Order G024012532 | Op Operation.AZ | M0 | 2024-11-01 04:30 -> 2024-11-01 07:50
Order G024013065 | Op Operation.HR | M0 | 2024-11-01 05:00 -> 2024-11-01 10:00
Order G024012526 | Op Operation.AZ | M0 | 2024-11-01 07:50 -> 2024-11-01 11:10
Order G024013186 | Op Operation.HR | M0 | 2024-11-01 10:00 -> 2024-11-01 18:30
Order G024012532 | Op Operation.LT | M0 | 2024-11-01 11:50 -> 2024-11-01 16:50
Order G024013065 | Op Operation.AZ | M0 | 2024-11-01 12:00 -> 2024-11-01 18:40
Order G024012526 | Op Operation.LT | M0 | 2024-11-01 16:50 -> 2024-11-01 21:50
Order G024013185 | Op Operation.HR | M0 | 2024-11-01 18:30 -> 2024-11-02 01:30


绘制库存变化图...



绘制拖期变化图...
